<a href="https://colab.research.google.com/github/Neulvo/TIL/blob/master/11_Prac_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#11장 감정분석(Sentiment Analysis)



#11-1 사전기반 감정분석

In [1]:
#KnuSentiLex 다운로드
!wget https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/pos_pol_word.txt
!wget https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/neg_pol_word.txt

--2020-08-02 23:59:57--  https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/pos_pol_word.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85007 (83K) [text/plain]
Saving to: ‘pos_pol_word.txt’

pos_pol_word.txt    100%[===================>]  83.01K  --.-KB/s    in 0.03s   

2020-08-02 23:59:57 (2.56 MB/s) - ‘pos_pol_word.txt’ saved [85007/85007]

--2020-08-02 23:59:58--  https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/neg_pol_word.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176542 (172K) [text/plain

In [2]:
!head -50 pos_pol_word.txt

출처
1) 표준국어대사전 뜻풀이
-> https://github.com/mrchypark/stdkor
2) 신조어
-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D
3) 이모티콘
-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98
4) SentiWordNet_3.0.0_20130122
-> http://sentiwordnet.isti.cnr.it/
5) SenticNet-5.0
-> http://sentic.net/
6) 감정단어사전0603
-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 
7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원


단어 수 : 4863
극성 : 긍정

(-;
(^^)
(^-^)
(^^*
(^_^)
(^o^)
*^^*
/^o^\
:(
:'-(
:-(
:-)
:|
;
;)
;-)
^///^
^^
^^//
<3
=)
XD
ㄱㅅ
ㄱㅇㄷ
가격이 싸다
가까이 사귀어
가까이하다
가다듬어
가다듬어 수습하는
가다듬어 수습하다
가다듬어 정하다


In [3]:
dct = {}

# 긍정단어 조회
with open('pos_pol_word.txt', 'r')as f:
  dct['pos'] = f.read().split('\n')[19:]

# 부정단어 조회
with open('neg_pol_word.txt', 'r')as f:
  dct['neg'] = f.read().split('\n')[19:]

print(dct['pos'][:10])
print(dct['neg'][:10])

['(-;', '(^^)', '(^-^)', '(^^*', '(^_^)', '(^o^)', '*^^*', '/^o^\\', ':(', ":'-("]
['가난', '가난뱅이', '가난살이', '가난살이하다', '가난설음', '가난에', '가난에 쪼들려서', '가난하게', '가난하고', '가난하고 어렵다']


In [4]:
txt = "코로나19의 여파로 전 세계 교사들과 학생들 모두 혼란스러운 시기를 보내고 있습니다. 초유의 상황에서도, 학생들을 위해 최선을 다하려는 선생님들의 노력이 계속되고 있는데요. 어려움 속에서, 선생님들은 어떤 방법으로 사랑을 전하고 있을까요? 뉴스G에서 전해드립니다. [리포트] 중국 저장성의 한 산골 마을. 초등학교 교사 왕진량 씨는 지난 2월 말부터, 매일 새벽 다섯 시면 부지런히 집을 나섭니다. 온라인 수업을 받을 수 없는, 깊은 산골 마을에 살고 있는 학생들을 찾아가기 위해서인데요. 선생님이 하루 동안 이동하는 거리는 대략 30km정도. 차도 없이 도보로 네 개의 마을을 돌아다니며 학생들을 만납니다. 매일 이어지는 강행군이지만, 오로지 아이들의 학습이 중단되어서는 안 된다는 생각뿐입니다. 혹시 모를 사태에 대비해 학생들과의 접촉은 최대한 줄입니다. 숙제를 내주고, 검사 후에 모르는 문제를 알려주는 식으로 일대일 수업을 진행하고 있는데요. 아이들에게 배우는 즐거움이 얼마나 소중한 것인지 잘 알기에, 선생님은 이렇게라도 수업을 할 수 있다는 데서 행복을 느낍니다. 영국의 한 초등학교 교사인 젠 포울스 씨는 매일 아침, 무거운 짐을 앞 뒤, 양 옆으로 짊어지고 씩씩하게 발걸음을 옮깁니다. 코로나19로 학교가 문을 닫은 뒤, 형편이 어려운 학생들을 위해 매일 78인분의 점심 도시락을 배달하고 있는데요. 선생님이 재직 중인 초등학교는 전체 학생의 41퍼센트가 무상 급식 대상자이기 때문입니다. 도시락의 무게는 18kg, 걸어야 하는 거리는 8km에 달하지만 기다리는 학생들을 생각하며 지치지 않고 발걸음을 재촉합니다. 학생들은 창문을 통해서 반갑게 인사하기도 하고, 선생님이 볼 수 있게 감사 메시지를 붙여 놓기도 하는데요. 선생님이 정성껏 준비해 손수 배달한 사랑의 도시락. 봉쇄된 도시의 굶주린 아이들에게 소중한 한 끼 식사, 그 이상의 의미가 되고 있습니다. 미국 사우스다코타 주의 중학교 수학 교사인 크리스 와바 씨는, 커다란 화이트보드를 들고 학생의 집을 찾았습니다. 온라인 수업 후, 학생에게 이메일로 방정식 풀이법에 대한 질문을 받았기 때문인데요. 이메일로 답변해주는 것보다 직접 풀이 과정을 보여주는 게 낫다는 생각이었죠. 깜짝 놀란 학생을 마주한 채, 선생님은 현관문 앞에서 열정적으로 문제를 풀기 시작했습니다. 이 열정적인 강의는, 학생이 풀이법을 완벽히 이해할 때까지 이어졌는데요. 바이러스는 전 세계 교실에 혼란을 불러왔지만, 선생님들의 노력은 저마다의 방식으로 계속되고 있습니다. 어려움 속에서도 학생들을 위해 안간힘을 쓰고 있는 모든 선생님들에게, 응원과 박수를 함께 보냅니다."

In [5]:
def analye_sentiment(txt, dct):
  pos =[]
  neg =[]

  for p in dct['pos']:
    if p in txt:
      pos.append(p)
  for p in dct['neg']:
    if p in txt:
      neg.append(p)

  return (len(pos)/(len(pos)+len(neg)),pos), (len(neg)/(len(pos)+len(neg)),neg)

In [6]:
pos, neg=analye_sentiment(txt, dct)
print(pos)
print(neg)

(0.6470588235294118, ['낫다', '대상', '사랑을', '씩씩하게', '열정', '운', '잘', '정', '정성껏', '함께', '감사', '복을', '부지런', '부지런히', '사랑', '사랑의', '소중한', '즐거움', '즐거움이', '행복', '행복을', ''])
(0.35294117647058826, ['방정', '어려운', '어려움', '해', '혼란스러운', '화', '굶주린', '모르는', '바이러스', '우는', '짐', ''])


# 11-2 나이브 베이즈 분류기 활용 감정분석

## 2.1 간단 예제

In [7]:
text = [("I love you", "P"),
        ("love happy weekend", "P"),
        ("bore work job", "N"),
        ("I hate you", "N"),
        ("bore weekend", "N"),
        ("happy together", "P")]

In [8]:
X_train = [t[0] for t in text]
Y_train = [t[1] for t in text]
Y_train

['P', 'P', 'N', 'N', 'N', 'P']

In [9]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [10]:
print(clf.predict(count_vect.transform(["happy weekend"])))
print(clf.predict_proba(count_vect.transform(["happy weekend"])))

['P']
[[0.25 0.75]]


## 2.2 네이버 영화 리뷰 감정분석

In [11]:
# 네이버 영화 리뷰 다운로드
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

--2020-08-03 00:06:00--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2020-08-03 00:06:01--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M  26.5MB/s    in 0.7s    

2020-08-03 00:06:04 (26.5 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv('./ratings.txt', sep='\t').dropna()
df.head(10)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(df['document'], df['label'], random_state=0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf=MultinomialNB().fit(X_train_tfidf, y_train)
print(clf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [17]:
X_test[:10]

78888                     진짜 개웃김 ㅋㅋㅋ 이런 코드너무 좋다 언능 2나왔음 좋겠다
49058     고속버스타고 가는 길에 핸드폰으로 보다가 울었네요 ㅠㅠ 아 슬픈듯.. 개인적으로 진...
101711                           개성 없는 모방 일색의 유치하고 가벼운 오락물.
97260     상처가없는이들에겐 갖혀있는소녀의 저질액션 미친영화이겠으나 나에겐 꿈처럼시작해서 미련...
53590                               아 다시 보고 싶은데 디비디가 안나온다..
62095                                            김민종씨 멋집니다.
36349          무식하게 거대한 원양어선으로 대량포획하는 미국, 일본 등등 국가들은 반성해라!!
151288                                        콜럼버스는 사기꾼이었다!
88072                                           재미있어요 재미있어요
117532    일주일에 한번봐도 대충 내용이 감이 오는 뻔한 스토리젤 짜증나는건 민세현? 연기 못...
Name: document, dtype: object

In [21]:
X_new_counts = count_vect.transform(X_test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)
np.mean(predicted == y_test)

0.8322332893315733

In [19]:
predicted[:10]

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 0])

In [20]:
y_test[:10]

78888     1
49058     1
101711    0
97260     1
53590     1
62095     1
36349     1
151288    0
88072     1
117532    0
Name: label, dtype: int64